In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import re

In [2]:
from tensorflow.keras.optimizers import Adam


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Carga y pre-procesamiento de los datos

In [5]:
def load_data():
    url = "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
    df = pd.read_csv(url)
    return df

In [6]:
# Función para equilibrar el dataset
def balance_dataset(df):
    # Contar la cantidad de muestras por clase
    counts = df['label'].value_counts()
    
    # Encontrar la clase mayoritaria
    major_class = counts.idxmax()
    
    # Encontrar la cantidad de muestras de la clase mayoritaria
    major_count = counts.max()
    
    # Crear un nuevo dataframe vacío para almacenar las muestras equilibradas
    balanced_df = pd.DataFrame(columns=df.columns)
    
    # Iterar sobre cada clase y agregar muestras al nuevo dataframe
    for label, count in counts.items():
        if label == major_class:
            balanced_df = pd.concat([balanced_df, df[df['label'] == label]])
        else:
            # Calcular el número de muestras a agregar para equilibrar
            num_samples_to_add = major_count - count
            
            # Seleccionar aleatoriamente muestras de la clase minoritaria
            samples_to_add = df[df['label'] == label].sample(num_samples_to_add, replace=True)
            
            # Agregar las muestras al nuevo dataframe
            balanced_df = pd.concat([balanced_df, samples_to_add])
    
    return balanced_df.reset_index(drop=True)

In [7]:
def clean_text(text):
    text = re.sub(r'@\w+', '', text)  # eliminar menciones
    text = re.sub(r'#\w+', '', text)  # eliminar hashtags
    text = re.sub(r'http\S+', '', text)  # eliminar URLs
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # eliminar caracteres especiales
    text = re.sub(r'\s+', ' ', text)  # eliminar espacios múltiples
    return text.strip()

In [8]:
df = load_data()
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [9]:
df_clean = df.copy()
# Limpiar el tweet
df_clean['tweet'] = df_clean['tweet'].apply(clean_text)
df_clean['tweet'] = df_clean['tweet'].str.lower()  # convertir a minúsculas
df_clean['tweet'] = df_clean['tweet'].str.replace(r'\d+', '', regex=True)  # eliminar números
df_clean['tweet'] = df_clean['tweet'].str.replace(r'\s+', ' ', regex=True)  # eliminar espacios múltiples

In [10]:
df_clean.head()

,id,label,tweet
0,1,0,when a father is dysfunctional and is so selfi...
1,2,0,thanks for credit i cant use cause they dont o...
2,3,0,bihday your majesty
3,4,0,i love u take with u all the time in ur
4,5,0,factsguide society now


In [11]:
df_clean = balance_dataset(df_clean)

# Tokenización

In [12]:
# Tokenización
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df_clean['tweet'])

sequences = tokenizer.texts_to_sequences(df_clean['tweet'])
padded = pad_sequences(sequences, maxlen=50, padding='post')
padded = np.array(padded, dtype='int32')

# División del conjunto de datos: Train, test, validate

In [13]:
# Datos y etiquetas
X = padded
y = np.array(df_clean['label'])

In [14]:
# Dividir entre train y temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Dividir X_temp entre validación y test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)


In [28]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape , X_val.shape, y_val.shape

((40038, 50), (8580, 50), (40038, 1), (8580,), (8580, 50), (8580,))

# Creación del modelo RNN

In [16]:
vocab_size = 10000
embedding_dim = 64
input_length = 50

In [17]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=input_length),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Para clasificación binaria
])

c:\Users\User\OneDrive\Universidad Central\Deep Learning\An-alisis-de-Sentimiento-en-Tweets-con-LSTM\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


# Copilación y entrenamiento

In [26]:
y_train = y_train.reshape(-1, 1)

In [27]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train
    # X_train, y_train,
    # validation_data=(X_val, y_val),
    # epochs=5,
    # batch_size=32,
)


ValueError: Invalid dtype: object

-----------------------------------------------

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
vocab_size = 10000
embedding_dim = 64
input_length = 50

model = model_rnn.build_rnn(vocab_size, embedding_dim, input_length)
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()


history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=32
)

model.save("../models/rnn_model.h5")

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

import data_loader
import model_rnn
import evaluate
import utils  # si lo usas para métricas o visualización


import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Cargar datos
df = data_loader.load_data()

# Limpiar texto
df['tweet'] = df['tweet'].apply(utils.clean_text)

# Tokenización
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['tweet'])

sequences = tokenizer.texts_to_sequences(df['tweet'])
padded = pad_sequences(sequences, maxlen=50, padding='post')

# Datos y etiquetas
X = padded
y = np.array(df['label'])

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from tensorflow.keras.optimizers import Adam

vocab_size = 10000
embedding_dim = 64
input_length = 50

model = model_rnn.build_rnn(vocab_size, embedding_dim, input_length)
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()


history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=32
)

model.save("../models/rnn_model.h5")


from evaluate import evaluate_model

y_pred = model.predict(X_test)
y_pred_labels = (y_pred > 0.5).astype("int32")

evaluate_model(y_test, y_pred_labels)
